In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import log
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
iris_dataset = load_iris()
data = iris_dataset['data']
label = iris_dataset['target']
label_name = iris_dataset['target_names']

In [3]:
# 计算信息熵
def Entropy(DataSet):
    n = len(DataSet)
    label_counts = {}
    
    for item in DataSet:
        label_current = item[-1]
        if label_current not in label_counts.keys():
            label_counts[label_current] = 0
        label_counts[label_current] += 1
        ent = 0.0
        for key in label_counts:
            prob = label_counts[key]/n
            ent -= prob*log(prob,2)
    return ent

In [4]:
class Node:
    def __init__(self, dimension, threshold, isLeaf, left, right, species):
        self.dimension = dimension  # 划分维度
        self.threshold = threshold  # 划分阈值
        self.isLeaf = isLeaf  # 是否是叶节点
        self.left = left  # 左支（叶节点时为None）
        self.right = right  # 右支（叶节点时为None）
        self.species = species  # 分类（如果是叶节点）

In [5]:
def get_gini(label):
    """
    计算GINI值
    :param label: 数组，里面存的是分类
    :return: 返回Gini值
    """
    gini = 1
    dic = {}
    for target in label:
        if target in dic.keys():
            dic[target] += 1
        else:
            dic[target] = 1
    for value in dic.values():
        tmp = value / len(label)
        gini -= tmp * tmp
    return gini

In [6]:
def get_gini_index_min(feature, label, dimension):
    """
    获取某个维度的最小GiniIndex
    :param feature: 所有属性list
    :param label: 标记list
    :param dimension: 维度(从0开始)
    :return: gini_index(最小GiniIndex)  threshold(对应阈值)
    """
    attr = feature[:, dimension]
    gini_index = 1
    threshold = 0
    attr_sort = sorted(attr)
    candicate_thre = []
    # 寻找候选阈值
    for i in range(len(attr_sort) - 1):
        tmp = (attr_sort[i] + attr_sort[i + 1]) / 2
        if tmp not in candicate_thre:
            candicate_thre.append(tmp)
    # 寻找最小GiniIndex
    for thre_tmp in candicate_thre:
        index_small_list = [index for index in range(len(feature)) if attr[index] < thre_tmp]
        label_small_tmp = label[index_small_list]
        index_large_list = [index for index in range(len(feature)) if attr[index] >= thre_tmp]
        label_large_tmp = label[index_large_list]
        gini_index_tmp = get_gini(label_small_tmp) * len(label_small_tmp) / len(attr) + get_gini(label_large_tmp) * len(
            label_large_tmp) / len(attr)
        if gini_index_tmp < gini_index:
            gini_index = gini_index_tmp
            threshold = thre_tmp
    print(gini_index, threshold)
    return gini_index, threshold

In [7]:
def find_dimension_by_GiniIndex(feature, label):
    """
    寻找划分维度
    :param feature: 所有属性list
    :param label: 标记list
    :return: gini_index, threshold, dimension
    """
    dimension = 0
    threshold = 0
    gini_index_min = 1
    for d in range(len(feature[1])):
        gini_index, thre = get_gini_index_min(feature, label, d)
        if gini_index < gini_index_min:
            gini_index_min = gini_index
            dimension = d
            threshold = thre
    print(gini_index, threshold, dimension)
    return gini_index, threshold, dimension

In [8]:
def devide_by_dimension_and_thre(feature, label, threshold, dimension):
    """
    根据阈值和维度来划分数据集，返回小集和大集
    :param feature: 所有属性list
    :param label: 标记list
    :param threshold: 划分阈值
    :param dimension: 划分维度
    :return: feature_small, label_small, feature_large, label_large
    """
    attr = feature[:, dimension]
    index_small_list = [index for index in range(len(feature)) if attr[index] < threshold]
    feature_small = feature[index_small_list]
    label_small = label[index_small_list]
    index_large_list = [index for index in range(len(feature)) if attr[index] >= threshold]
    feature_large = feature[index_large_list]
    label_large = label[index_large_list]
    return feature_small, label_small, feature_large, label_large


def build_tree(feature, label):
    """
    递归构建决策树
    :param feature: 所有属性list
    :param label: 标记list
    :return: 决策树的根Node节点
    """
    if len(label) > 1:
        gini_index, threshold, dimension = find_dimension_by_GiniIndex(feature, label)
        if gini_index == 0:  # gini_index = 0，说明全都是同一种类型，就是叶节点
            return Node(dimension, threshold, True, None, None, label[0])
            print('end')
        else:
            # gini_index != 0，说明还不纯，继续划分，递归构建左支和右支
            feature_small, label_small, feature_large, label_large = devide_by_dimension_and_thre(feature, label,
                                                                                                  threshold,
                                                                                                  dimension)
            left = build_tree(feature_small, label_small)
            right = build_tree(feature_large, label_large)
            return Node(dimension, threshold, False, left, right, None)
    else:
        # 如果只有一个数据，直接是叶节点
        return Node(None, None, True, None, None, label[0])

In [9]:
build_tree(feature=data, label=label)

0.43890633176347466 5.45
0.5397432831061151 3.3499999999999996
0.3333333333333333 2.45
0.3333333333333333 0.8
0.3333333333333333 2.45 2
0.0 4.35
0.0 2.5999999999999996
0.0 1.05
0.0 0.1
0.0 4.35 0
0.3931313131313131 6.15
0.4644444444444445 2.45
0.12646464646464653 4.75
0.1103059581320451 1.75
0.1103059581320451 1.75 3
0.1369671558350803 7.1
0.1634085213032582 2.6500000000000004
0.0856481481481482 4.95
0.14957264957264954 1.35
0.14957264957264954 4.95 2
0.020833333333333332 4.95
0.038690476190476185 2.55
0.03947368421052637 4.45
0.0 1.65
0.0 1.65 3
0.39999999999999997 6.95
0.3333333333333333 2.6500000000000004
0.3333333333333333 5.35
0.22222222222222224 1.55
0.22222222222222224 1.55 3
0.0 6.05
0.0 2.4000000000000004
0.0 5.05
0.0 1.45
0.0 6.05 0
0.0 6.95
0.3333333333333333 2.85
0.0 5.449999999999999
0.3333333333333333 1.65
0.3333333333333333 6.95 0
0.0 6.35
0.0 2.85
0.0 5.05
0.0 1.65
0.0 6.35 0
0.03726708074534162 5.95
0.04037267080745343 3.1500000000000004
0.02898550724637681 4.85
0.0398

In [10]:
feature_train, feature_test, target_train, target_test = train_test_split(data, label, random_state=10)

In [11]:
def predict(root: Node, feature_line):
    """
    使用该方法进行预测
    :param root: 决策树根节点
    :param feature_line: 需要预测的属性值
    :return: 预测结构 label
    """
    node = root
    while not node.isLeaf:
        if feature_line[node.dimension] < node.threshold:
            node = node.left
        else:
            node = node.right
    return node.species


def score(root, feature, label):
    """
    模型得分评估
    :param root: 决策树根节点
    :param feature: 测试集属性list
    :param label: 测试集标记list
    :return: 正确率
    """
    correct = 0
    for index in range(len(feature)):
        type = predict(root, feature[index])
        if type == label[index]:
            correct += 1
    print('correct rate is', correct / len(feature))
    
    
res = build_tree(feature_train, target_train)
score(res, feature_test, target_test)

0.4491767353707652 5.55
0.5163908246225319 3.3499999999999996
0.3253424657534246 2.45
0.3253424657534246 0.8
0.3253424657534246 2.45 2
0.0 4.35
0.0 2.5999999999999996
0.0 1.05
0.0 0.1
0.0 4.35 0
0.3975692656298281 6.25
0.45676563565362915 2.95
0.1254385232208486 4.75
0.12466783396848286 1.75
0.12466783396848286 1.75 3
0.17105263157894746 4.95
0.1739130434782609 2.8499999999999996
0.11419457735247202 4.95
0.16620498614958445 1.35
0.16620498614958445 4.95 2
0.030303030303030304 4.95
0.05454545454545453 2.55
0.05555555555555559 4.45
0.0 1.65
0.0 1.65 3
0.3 6.5
0.2666666666666667 2.6500000000000004
0.4 5.35
0.0 1.55
0.0 1.55 3
0.04571428571428569 5.95
0.051948051948051986 3.1500000000000004
0.02857142857142857 4.85
0.051948051948051986 1.85
0.051948051948051986 4.85 2
0.0 5.95
0.0 3.1
0.5 4.8
0.5 1.8
0.5 5.95 0
0.0 5.65
0.0 2.5
0.0 4.9
0.0 1.8
0.0 5.65 0
correct rate is 0.9736842105263158
